# KdV Uncertainty output

## Plots

Produce the following plots:
 - Example a0 and density profiles for one time step
 - Example of a time-series all A(t) for 1, 10, 500 runs
 - Timeseries of 'max_amplitude' all runs
 - Timeseries of normalized 'max_amplitude' all runs
 
---
Other plots for the paper/talks:
 - *Example of scenario A(x,t)* requires running a simulation...

In [ ]:
import h5py
from glob import glob
import numpy as np

from dask import array as da
from dask import delayed

from iwaves.utils import isw

import matplotlib.pyplot as plt
from tqdm import tqdm # progress bar

%matplotlib inline

# Test loading some data

Data are stored in individual hdf5 files for each step (1473) and each sample (500)

In [15]:
#sorted(glob("../../shiny_dashboard/data/timepoint-1/all_amplitudes/*.h5"))

def get_file_name(timepoint, sample):
    return "../../kdv_solutions_azure/shiny_dashboard/data/timepoint-%d/all_amplitudes/2018-07-24--13-00-34_timepoint-%d_sample-%d_output.h5"%(timepoint,timepoint,sample)

def get_all_file(timepoint):
    return sorted(glob("../../kdv_solutions_azure/shiny_dashboard/data/timepoint-%d/all_amplitudes/*.h5"%(timepoint)))

def get_summary_file(timepoint):
    return sorted(glob("../../kdv_solutions_azure/shiny_dashboard/data/timepoint-%d/*.h5"%(timepoint)))[0]
#get_file_name(1,0)
#get_all_file(1)
get_summary_file(1)

'../../kdv_solutions_azure/shiny_dashboard/data/timepoint-1/2018-07-24--13-00-34_slim-output.h5'

In [16]:
h5 = h5py.File(get_summary_file(1),'r')
list(h5.keys())

['a0_samples', 'beta_samples', 'max_amplitude', 'timepoint']

In [21]:
h5['a0_samples'][:].shape

(500,)

In [20]:
h5['beta_samples'][:].shape

(6, 500)

In [22]:
h5['max_amplitude'].value.shape

(500,)

In [ ]:
h5['amplitudes'].shape

# Data loading tools

In [29]:
#@delayed(nout=1)
def load_h5_step_old(varname, timepoint):
    nsamples = 500

    a0 = []
    for ii in range(nsamples):
        try:
            h5 = h5py.File(get_file_name(timepoint,ii),'r')
            a0.append(da.from_array( h5[varname].value, chunks=-1))
            h5.close()
        except:
            continue
            #print('Failed to load sample %d...'%ii)
    
    if len(a0) == 0:
        print('No files found for step %d'%timepoint)
    a0_da = da.stack(a0)

    return a0_da

def load_h5_step(varname, timepoint):
    files = get_all_file(timepoint)
    nsamples = len(files)

    a0 = []
    for ii in range(nsamples):
        h5 = h5py.File(files[ii],'r')
        a0.append(da.from_array( h5[varname].value, chunks=-1))
        h5.close()

    a0_da = da.stack(a0)

    return a0_da

@delayed(nout=1)
def load_h5_alltime(varname, maxpoints):

    a0 = []
    for ii in range(maxpoints):
        a0_da = load_h5_step(varname, ii)
        a0.append(a0_da)
        
    #a0_t = da.stack(a0, axis=0)
    
    return a0

def return_a0_beta(timestep, ensemble):
    h5 = h5py.File(get_summary_file(timestep),'r')
    a0 = h5['a0_samples'][ensemble]
    beta = h5['beta_samples'][:,ensemble]
    h5.close()
    
    return a0, beta

def return_max_ensemble(timestep):
    h5 = h5py.File(get_summary_file(timestep),'r')
    a0 = h5['max_amplitude'][:]
    h5.close()
    
    return np.argwhere(np.abs(a0) == np.abs(a0).max())[0,0]
#amp, beta, a0 = load_h5_step(1)

return_max_ensemble(1)

160

In [ ]:
tstep = 50
a0 = load_h5_step('a0_sample', tstep).compute()
beta = load_h5_step('beta_samples', tstep).compute()
amax = load_h5_step('max_amplitude', tstep).compute()

#a0 = load_h5_alltime('beta_samples',10)
#a0 = a0.compute()
#beta = beta.compute()

# Plot of some initial conditions for one scenario

In [ ]:
def double_tanh(beta, z):
    
    return beta[0] - beta[1]*(np.tanh((z+beta[2])/beta[3])
                + np.tanh((z+beta[4])/beta[5]))

In [ ]:
# Depth
zmin = -250
zout = np.linspace(zmin,0,100)

In [ ]:
plt.figure()
plt.hist(a0.compute(),bins=25, color='0.5', edgecolor='k')
plt.xlabel('$a_0$ [m]')
plt.ylabel('Count')

In [ ]:
# Plot the density profile
plt.figure(figsize=(5,6))
nsamples = 500
for rand_loc in range(0, nsamples):
    rhotmp = double_tanh(beta[rand_loc,:], zout)
    plt.plot(rhotmp, zout, '0.5', lw=1, alpha=0.02)
    
#plt.plot(rhomean*RHO0, zout, 'k--',) # Mean fit

plt.grid(b=True)
plt.ylim(zmin,0 )
plt.ylabel('Depth [m]')
plt.xlabel(r'$\rho$ [kg m$^{-3}$]')
#plt.title(rho.time.values[tstep])

In [ ]:
# Calculate alpha, cn for a time step

def calc_alpha(beta, zout, nsamples=500, mode=0):
    
    alpha_ens = np.zeros((nsamples,))
    c_ens = np.zeros((nsamples,))

    dz = np.diff(zout).mean()
    
    for ii in range(nsamples):
        rhotmp = double_tanh(beta[ii,:], zout)

        N2 = -9.81/1000*np.gradient(rhotmp,dz)

        phi,cn = isw.iwave_modes(N2, dz)

        phi_1 = phi[:,mode]
        phi_1 =phi_1 / np.abs(phi_1).max()
        phi_1 *= np.sign(phi_1.sum())

        alpha = isw.calc_alpha(phi_1, cn[mode],N2,dz)

        alpha_ens[ii] = alpha
        c_ens[ii] = cn[mode]
        
    return alpha_ens, c_ens

In [ ]:
alpha, cn = calc_alpha(beta, zout)

T = 12.42*3600.
omega = 2*np.pi / T
Ls = cn**2. / (np.abs(alpha)*a0*omega) 
Ls

In [ ]:
plt.figure()
plt.subplot(131)
plt.hist(alpha,bins=np.arange(-0.012,0.01205,0.0005), color='0.5', edgecolor='k')
plt.xlabel(r'$\alpha$ []')
plt.ylabel('Count')

plt.subplot(132)
plt.hist(cn,bins=np.arange(0.9,1.3,0.005), color='0.5', edgecolor='k')
plt.xlabel(r'$c$ [m s]')

plt.subplot(133, xscale='log')
plt.hist(Ls,bins=np.arange(10e3,1000000,10e3), color='0.5', edgecolor='k')
plt.xlabel(r'$Ls$ [m]')
#plt.ylabel('Count')

In [ ]:
plt.figure()
plt.subplot(121)
plt.hist(amax,bins=np.arange(-100,105,5), color='0.5', edgecolor='k')
plt.xlabel(r'$A_{max}$ [m]')
plt.ylabel('Count')

plt.subplot(122)
plt.hist(amax/a0,bins=np.arange(-3.5,3.55,0.05), color='0.5', edgecolor='k')
plt.xlabel(r'$A_{max}/a0$')
plt.xlim(-3.5,3.5)


In [ ]:
L = 1e5 # Location of output point

plt.semilogx(Ls/L, amax/a0, 'k.')
plt.xlim(1e-1,10)
plt.ylim(-4,4)
plt.grid(b=True)

plt.ylabel(r'$A_{max}/a0$')
plt.xlabel('$L_s / L$')


In [ ]:
# Plot the amplitudes for all runs
amp = load_h5_step('amplitudes', tstep).compute()

dt = 20
nt = amp.shape[1]
time = np.arange(0,nt)*dt

In [ ]:
plt.figure(figsize=(5,4))
nsamples = 500
for rand_loc in range(0, nsamples):
    plt.plot(time/T, amp[rand_loc,:], '0.5', lw=0.5, alpha=0.05)
    
#plt.plot(rhomean*RHO0, zout, 'k--',) # Mean fit

plt.grid(b=True)
plt.ylim(-100,100 )
plt.xlim(1.5,3)
plt.ylabel('Amplitude [m]')
plt.xlabel(r'time [cycles]')

In [ ]:
# Get amax/a0 for all steps
nt = 500
nsamples = 500
amax_t = np.zeros((nsamples,nt))
a0_t = np.zeros((nsamples,nt))
alpha_t = np.zeros((nsamples,nt))
cn_t = np.zeros((nsamples,nt))
beta_t = np.zeros((nt, nsamples,6))

for ii in tqdm(range(0,nt)):
    #try:
    a0_tmp = load_h5_step('a0_sample', ii+1).compute()
    amax_tmp = load_h5_step('max_amplitude', ii+1).compute()
    beta_tmp = load_h5_step('beta_samples', ii+1).compute()
    
    ns = a0_tmp.shape[0]
    if ns < 500:
        print('Only found %d sample for step %d'%(ns,ii))
    alpha_tmp, c_tmp = calc_alpha(beta_tmp, zout, nsamples=ns, mode=0)    
    
    a0_t[0:ns,ii] = a0_tmp
    amax_t[0:ns,ii] = amax_tmp
    alpha_t[0:ns,ii] = alpha_tmp
    cn_t[0:ns,ii] = c_tmp
    
    beta_t[ii,0:ns,:] = beta_tmp



    #    ngood = a0_tmp.shape[0]
    #    
    #    a0_t[0:ngood,ii] = a0_tmp
    #    amax_t[0:ngood:,ii] = amax_tmp
    #except:
    #    print('Failed to load step %d...'%ii)



In [ ]:
alpha_tmp

In [ ]:
A_t = amax_t/a0_t

plt.plot(A_t.min(axis=0))

In [ ]:
a0_tmp = load_h5_step('a0_sample', 50).compute()
